In [ ]:
#Only run in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/flywire

In [ ]:
import numpy as np
import scipy as sp
import cupy as cp
from utility import get_best_connected, invert, score, get_highest_out_degrees
from data_loader import M,F, benchmark_mapping
rng = np.random.default_rng(0)

In [1]:
def make_best_swap(G,H, mapping, u, scores_by_vertex,cprng=None):

    m = G.shape[0]
    n = H.shape[0]

    row = G[u,:]
    rnz = cp.flatnonzero(row)
    row = row[rnz]
    col = G[:,u]
    cnz = cp.flatnonzero(col)
    col = col[cnz]

    X = H[mapping[:,None],mapping[None,rnz]].reshape((n,len(rnz)))
    Y = H[mapping[cnz,None],mapping[None,:]].reshape((len(cnz),n))

    u_scores = cp.minimum(row[None,:],X).sum(axis=1) + cp.minimum(col[:,None],Y).sum(axis=0)

    f_u = mapping[u].copy()

    row = H[f_u,mapping]
    rnz = np.flatnonzero(row)
    row = row[rnz]
    col = H[mapping,f_u]
    cnz = np.flatnonzero(col)
    col = col[cnz]

    X = G[:,rnz[None,:]].reshape((m,len(rnz)))
    Y = G[cnz[:,None],:].reshape((len(cnz),m))

    f_u_scores = cp.minimum(row[None,:],X).sum(axis=1) + cp.minimum(col[:,None],Y).sum(axis=0)

    scores = u_scores + f_u_scores + np.minimum(G[u,:],H[mapping,f_u])+np.minimum(G[:,u],H[f_u,mapping])
    scores -= scores_by_vertex
    scores -= scores_by_vertex[u]
    scores += cp.minimum(G[u,:],H[f_u,mapping])+cp.minimum(G[:,u],H[mapping,f_u])

    if cprng is None:
        w = scores.argmax()
    else:
        max_value = scores.max()
        max_indices=cp.flatnonzero(scores==max_value)
        w = max_indices[cprng.integers(len(max_indices))]
        
    if scores[w] > 0:

        f_w = mapping[w].copy()

        scores_by_vertex += cp.minimum(G[:,u],H[mapping,f_w])+cp.minimum(G[u,:],H[f_w,mapping])+cp.minimum(G[:,w],H[mapping,f_u])+cp.minimum(G[w,:],H[f_u,mapping])
        scores_by_vertex -= cp.minimum(G[:,u],H[mapping,f_u])+cp.minimum(G[u,:],H[f_u,mapping])+cp.minimum(G[:,w],H[mapping,f_w])+cp.minimum(G[w,:],H[f_w,mapping])

        scores_by_vertex[u] = u_scores[w]+ cp.minimum(G[u,w],H[f_w,f_u])+cp.minimum(G[w,u],H[f_u,f_w])
        scores_by_vertex[w] = f_u_scores[w]+ cp.minimum(G[u,w],H[f_w,f_u])+cp.minimum(G[w,u],H[f_u,f_w])

        mapping[u] = f_w
        mapping[w] = f_u

        return scores[w]

    else:
        return 0

def get_scores_by_vertex(G,H,mapping):

    m = G.shape[0]
    scores = cp.minimum(G,H[mapping[:,None],mapping[None,:]])
    return scores.sum(axis=0)+scores.sum(axis=1)

def move_or_swap(G,H, mapping, u, scores_by_vertex, usage, penalty_gradient):

    m = G.shape[0]
    n = H.shape[0]
    f_u = mapping[u].copy()

    row = G[u,:]
    rnz = cp.flatnonzero(row)
    row = row[rnz]
    col = G[:,u]
    cnz = cp.flatnonzero(col)
    col = col[cnz]

    X = H[:,mapping[None,rnz]].reshape((n,len(rnz)))
    Y = H[mapping[cnz,None],:].reshape((len(cnz),n))

    base_scores = cp.minimum(row[None,:],X).sum(axis=1) + cp.minimum(col[:,None],Y).sum(axis=0)
    u_scores = base_scores[mapping]

    move_scores = base_scores - base_scores[f_u]
    move_scores -= penalty_gradient(usage)
    move_scores += penalty_gradient(usage[f_u]-1)
    move_scores[f_u] = 0

    row = H[f_u,mapping]
    rnz = cp.flatnonzero(row)
    row = row[rnz]
    col = H[mapping,f_u]
    cnz = cp.flatnonzero(col)
    col = col[cnz]

    X = G[:,rnz[None,:]].reshape((m,len(rnz)))
    Y = G[cnz[:,None],:].reshape((len(cnz),m))

    f_u_scores = cp.minimum(row[None,:],X).sum(axis=1) + cp.minimum(col[:,None],Y).sum(axis=0)

    swap_scores = u_scores + f_u_scores + cp.minimum(G[u,:],H[mapping,f_u])+cp.minimum(G[:,u],H[f_u,mapping])
    swap_scores -= scores_by_vertex
    swap_scores -= scores_by_vertex[u]
    swap_scores += cp.minimum(G[u,:],H[f_u,mapping])+cp.minimum(G[:,u],H[mapping,f_u])

    x = move_scores.argmax()
    w = swap_scores.argmax()
    if swap_scores[w] > 0 and swap_scores[w] >= move_scores[x]:

        f_w = mapping[w].copy()

        scores_by_vertex += cp.minimum(G[:,u],H[mapping,f_w])+cp.minimum(G[u,:],H[f_w,mapping])+cp.minimum(G[:,w],H[mapping,f_u])+cp.minimum(G[w,:],H[f_u,mapping])
        scores_by_vertex -= cp.minimum(G[:,u],H[mapping,f_u])+cp.minimum(G[u,:],H[f_u,mapping])+cp.minimum(G[:,w],H[mapping,f_w])+cp.minimum(G[w,:],H[f_w,mapping])

        scores_by_vertex[u] = u_scores[w]+ cp.minimum(G[u,w],H[f_w,f_u])+cp.minimum(G[w,u],H[f_u,f_w])
        scores_by_vertex[w] = f_u_scores[w]+ cp.minimum(G[u,w],H[f_w,f_u])+cp.minimum(G[w,u],H[f_u,f_w])

        mapping[u] = f_w
        mapping[w] = f_u

        return swap_scores[w]
    elif move_scores[x] > 0:

        scores_by_vertex += cp.minimum(G[:,u],H[mapping,x])+cp.minimum(G[u,:],H[x,mapping]) - cp.minimum(G[:,u],H[mapping,f_u])- cp.minimum(G[u,:],H[f_u,mapping])

        scores_by_vertex[u] = base_scores[x]

        mapping[u] = x
        usage[f_u] -= 1
        usage[x] += 1
        return base_scores[x]-base_scores[f_u]
    else:
        return 0

Hello World


In [ ]:
A=cp.asarray(M.todense())
B=cp.asarray(F.todense())
N=A.shape[0]
A[cp.arange(N),cp.arange(N)]=0
B[cp.arange(N),cp.arange(N)]=0

In [ ]:
G= B
H=A
mapping = cp.asarray(invert(benchmark_mapping))

In [ ]:
scores_by_vertex = get_scores_by_vertex(G,H,mapping)
score_track=scores_by_vertex.sum()//2
cprng = cp.random.default_rng()

for i in range(10):
  for j,u in enumerate(cp.asarray(rng.permutation(N))):#range(1319,N):
    improvement = make_best_swap(G,H,mapping,u,scores_by_vertex,cprng=cprng)
    score_track += improvement
    if j%20 == 0:
      print(f"{i=}, {j=}, {score_track=}",end="\n")

In [ ]:
cprng = cp.random.default_rng()

for i in range(10):

  scores_by_vertex = get_scores_by_vertex(G,H,mapping)
  score_track=scores_by_vertex.sum()//2
  for j,u in enumerate(cp.asarray(rng.permutation(N))):
    improvement = make_best_swap(G,H,mapping,u,scores_by_vertex,cprng=cprng)
    score_track += improvement
    if j%20 == 0:
      print(f"{i=}, {j=}, {score_track=}",end="\n")

  tmp=G
  G=H
  H=tmp
  mapping = invert(mapping)

In [ ]:
G=B
H=A

top_score = score(G,H,mapping,require_surjective=False)
usage = cp.zeros(H.shape[0],dtype=cp.int32)
cp.add.at(usage,mapping,1)
scores_by_vertex = get_scores_by_vertex(G,H,mapping)
score_track=scores_by_vertex.sum()//2

def penalty_gradient(usage):

   return 16*usage

for i in range(5):
  for j,u in enumerate(cp.asarray(rng.permutation(N))):
    improvement = move_or_swap(G,H,mapping,u,scores_by_vertex,usage,penalty_gradient=penalty_gradient)
    score_track += improvement
    if j%20==0:
      print(f"{i=}, {j=}, {score_track=}, unique={len(cp.unique(mapping))}",end="\n")

score(G,H,mapping,require_surjective=False), score(G,G,np.arange(N))